In [2]:
import itertools
import logging
from pathlib import Path

import awkward as ak
import click
import h5py
import numpy as np
import vector

from src.data.cms.convert_to_h5 import MIN_JETS, N_JETS, N_FJETS

vector.register_awkward()

logging.basicConfig(level=logging.INFO)

/Users/billyli/miniforge_x86_new/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
HIGGS_MASS = 125.0
PROJECT_DIR = Path(__file__).resolve().parents[2]
# precompute possible jet assignments lookup table
JET_ASSIGNMENTS = {}
for nj in range(MIN_JETS, N_JETS + 1):
    a = list(itertools.combinations(range(nj), 2))
    b = np.array([(i, j, k) for i, j, k in itertools.combinations(a, 3) if len(set(i + j + k)) == MIN_JETS])
    JET_ASSIGNMENTS[nj] = b

FJET_ASSIGNMENTS = {}

In [3]:
# a function that loads jets from hhh_test.h5
def load_jets(in_file):
    # load jets from the h5
    pt = ak.Array(in_file["INPUTS"]["Jets"]["pt"])
    eta = ak.Array(in_file["INPUTS"]["Jets"]["eta"])
    sinphi = ak.Array(in_file["INPUTS"]["Jets"]["sinphi"])
    cosphi = ak.Array(in_file["INPUTS"]["Jets"]["cosphi"])
    phi = np.arcsin(sinphi)
    btag = ak.Array(in_file["INPUTS"]["Jets"]["btag"])
    mass = ak.Array(in_file["INPUTS"]["Jets"]["mass"])
    mask = ak.Array(in_file["INPUTS"]["Jets"]["MASK"])

    # remove zero-padded jets
    pt = pt[mask]
    eta = eta[mask]
    sinphi = sinphi[mask]
    cosphi = cosphi[mask]
    phi = phi[mask]
    btag = btag[mask]
    mass = mass[mask]

    jets = ak.zip(
        {
            "pt": pt,
            "eta": eta,
            "sinphi": sinphi,
            "cosphi": cosphi,
            "phi": phi,
            "btag": btag,
            "mass": mass
        },
        with_name="Momentum4D",
    )
    
    return jets

In [26]:
# a function that loads fat jets from hhh_test.h5
def load_fjets(in_file):
     # load fatjets from h5
    fj_pt = ak.Array(in_file["INPUTS"]["BoostedJets"]["fj_pt"])
    fj_eta = ak.Array(in_file["INPUTS"]["BoostedJets"]["fj_eta"])
    fj_sinphi = ak.Array(in_file["INPUTS"]["BoostedJets"]['fj_sinphi'])
    fj_cosphi = ak.Array(in_file["INPUTS"]["BoostedJets"]["fj_cosphi"])
    fj_mass = ak.Array(in_file["INPUTS"]["BoostedJets"]["fj_mass"])
    fj_mask = ak.Array(in_file["INPUTS"]["BoostedJets"]["MASK"])

    # remove zero-padded jets
    fj_pt = fj_pt[fj_mask]
    fj_eta = fj_eta[fj_mask]
    fj_sinphi = fj_sinphi[fj_mask]
    fj_cosphi = fj_cosphi[fj_mask]
    fj_mass = fj_mass[fj_mask]

    fjets = ak.zip(
        {
            "fj_pt": fj_pt,
            "fj_eta": fj_eta,
            "fj_sinphi": fj_sinphi,
            "fj_cosphi": fj_cosphi,
            'fj_mass': fj_mass
        },
        with_name="Momentum4D"
    )
    
    return fjets

In [24]:
@click.command()
@click.option("--pred_file", '-pf', default=f"{str(Path.cwd().joinpath('predictions.h5'))}", help="Path of the output prediction file")
@click.option("--test_file", '-tf', default='-sf', help="Event file")

SyntaxError: unexpected EOF while parsing (1900233828.py, line 3)

In [27]:
in_file = h5py.File(test_file)
fjets = load_fjets(in_file)

<MomentumArray4D [[], [], ..., [{fj_pt: 323, ...}], []] type='42829 * var *...'>

In [17]:
pred_file = ""
test_file = "/Users/billyli/UCSD/hhh/data/delphes/v2/hhh_test.h5"
def main(pred_file, test_file):
    # load input test h5 file
    in_file = h5py.File(test_file)
    
    jets = load_jets(in_file)
    fjets = load_fjets(in_file)
    
    # select fjets based on pT and mass
    fjmass_cond = (fjets['fj_mass']>110) & (fjets['fj_mass']<140)
    fjpt_cond = fjets['fj_pt']>300
    fj_cond = fjmass_cond & fjpt_cond
    fjets_selected = fjets[fj_cond]
    
    # find ak4jets that matched to selected ak8jets (dR check)
    
    # remove overlapped ak4jets
    
    # assign ak4 jets to the remaining higgs by chi2
    
    # save all assignment to the h5file
    
    return
    
    

   


main(pred_file, test_file)


## code segments waiting

In [ ]:

    
    # just consider top-6 jets
    nj = 6
    mjj = (jets[:, JET_ASSIGNMENTS[nj][:, :, 0]] + jets[:, JET_ASSIGNMENTS[nj][:, :, 1]]).mass
    chi2 = ak.sum(np.square(mjj - HIGGS_MASS), axis=-1)
    chi2_argmin = ak.argmin(chi2, axis=-1)

    h1_bs = np.concatenate(
        (
            np.array(in_file["TARGETS"]["h1"]["b1"])[:, np.newaxis],
            np.array(in_file["TARGETS"]["h1"]["b2"])[:, np.newaxis],
        ),
        axis=-1,
    )
    h2_bs = np.concatenate(
        (
            np.array(in_file["TARGETS"]["h2"]["b1"])[:, np.newaxis],
            np.array(in_file["TARGETS"]["h2"]["b2"])[:, np.newaxis],
        ),
        axis=-1,
    )
    h3_bs = np.concatenate(
        (
            np.array(in_file["TARGETS"]["h3"]["b1"])[:, np.newaxis],
            np.array(in_file["TARGETS"]["h3"]["b2"])[:, np.newaxis],
        ),
        axis=-1,
    )

    # chi2 on fjets to find Higgs
    
    num_events = len(fj_pt) 
    bh1_b_pred = np.ones(shape=(num_events, 1), dtype=int)
    bh2_b_pred = np.ones(shape=(num_events, 1), dtype=int)*2
    bh3_b_pred = np.ones(shape=(num_events, 1), dtype=int)*3

    bh1_b = np.array(in_file["TARGETS"]["bh1"]["bb"])
    bh2_b = np.array(in_file["TARGETS"]["bh2"]["bb"])
    bh3_b = np.array(in_file["TARGETS"]["bh3"]["bb"])

    targets = [h1_bs, h2_bs, h3_bs, bh1_b, bh2_b, bh3_b]

    masks = np.concatenate(
        (
            np.array(in_file["TARGETS"]["h1"]["mask"])[np.newaxis, :],
            np.array(in_file["TARGETS"]["h2"]["mask"])[np.newaxis, :],
            np.array(in_file["TARGETS"]["h3"]["mask"])[np.newaxis, :],
            np.array(in_file["TARGETS"]["bh1"]["mask"])[np.newaxis, :],
            np.array(in_file["TARGETS"]["bh2"]["mask"])[np.newaxis, :],
            np.array(in_file["TARGETS"]["bh3"]["mask"])[np.newaxis, :]
        ),
        axis=0,
    )

    predictions = [
        JET_ASSIGNMENTS[nj][chi2_argmin][:, 0, :],
        JET_ASSIGNMENTS[nj][chi2_argmin][:, 1, :],
        JET_ASSIGNMENTS[nj][chi2_argmin][:, 2, :],
        bh1_b_pred,
        bh2_b_pred,
        bh3_b_pred,
    ]

    num_vectors = np.sum(mask, axis=-1).to_numpy() # number of unique objects in every event
    lines = 2 # how many lines are generated in the table
    results, jet_limits, clusters = evaluate_predictions(predictions, num_vectors, targets, masks, event_file, lines)
    display_table(results, jet_limits, clusters)
